In [ ]:
from typing import Literal
from tavily import TavilyClient
from langchain.chat_models import init_chat_model
from deepagents import create_deep_agent
from deepagents.backends import CompositeBackend, StateBackend, FilesystemBackend
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
tavily_client = TavilyClient()

def internet_search(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """Run a web search"""
    return tavily_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )

research_subagent = {
    "name": "research-agent",
    "description": "Used to run a web search for a single topic, such as facts, trends, and relevant information",
    "system_prompt": """You are a researcher. Your job is to gather information on a given topic using web search.
    Output format:
    - Summary (2-3 paragraphs)
    - Key findings (bullet points)
    - Sources (with URLs)

    Keep your response under 500 words to maintain clean context.
    """,
    "tools": [internet_search],
    "model": "openai:gpt-4o-mini",
}
subagents = [research_subagent]

In [3]:
main_agent_prompt = """
You are an experienced planner agent that can help the user with their ambitious personal projects.
You must think about the goal they are seeking to achieve, reason about the steps needed to get there, and create a detailed plan of action.

IMPORTANT: Always consider the user's specified time window when creating the plan. Adjust the granularity, pace, and milestones based on whether they have days, weeks, months, or years to achieve their goal.

The steps must be actionable and broken down into manageable tasks. Each task should have a clear objective and a suggested timeline that fits within the user's available time window.
Your plan should consider the up-to-date best practices. There is a web search tool available to you if you need to look up any information.
Use it as much as needed to ensure your plan is comprehensive and well-informed.

Finally, compile your findings into a well-structured markdown report that includes:
- Executive Summary
- Goal Overview
- Timeline Breakdown (based on user's time window)
- Detailed Action Steps with milestones
- Resources Needed
- Potential Challenges and Mitigation Strategies
- Success Metrics
"""

In [ ]:
composite_backend = lambda rt: CompositeBackend(
    default=StateBackend(rt),
    routes={
        "/memories/": FilesystemBackend(root_dir="path/to/your/project/deepagent/backend/memories", virtual_mode=True),
    },
)

model = init_chat_model("openai:gpt-5-mini")

agent = create_deep_agent(
    system_prompt=main_agent_prompt,
    subagents=subagents,
    model=model,
    backend=composite_backend
)

In [5]:
user_query = """
Hi, I am John. I want to start a start a bakery business that specializes in gluten-free pastries in London.
I want to do this in the next 6 months. I have already passed a training course on baking gluten-free goods.
Therefore, I have some basic knowledge of the field as well as the required certifications.
However, I don't know where to start with the business planning, market research, and other necessary steps.
Please create a detailed action plan for me within the next 6 months, including market research, business planning, marketing strategies, 
and any other relevant steps.
"""

In [6]:
result = agent.invoke({"messages": [{"role": "user", "content": user_query}]})

# Print the agent's response
print(result["messages"][-1].content)

Here’s a six‑month, step‑by‑step launch plan tailored for a gluten‑free pastry bakery in London. It’s staged to validate demand early, manage regulatory steps on time, and get you trading with a lean footprint before scaling.

Executive Summary
- Strategy: Start lean with a low‑risk market entry (pop‑ups/market stall/production kitchen + pre‑orders), build a loyal gluten‑free community, then scale to a permanent site or hybrid model once unit economics are validated.
- Differentiator: “Truly gluten‑free, exceptional taste” with rigorous allergen control and transparent labelling. Consider Coeliac UK Gluten Free Accreditation to build trust.
- Milestones (headline):
  - Month 1: Concept, research, and costing complete
  - Month 2: Product R&D finalized, brand + website live, suppliers secured
  - Month 3: Register food business (28+ days before trading), HACCP/SFBB complete, pre‑launch sampling
  - Month 4: Pilot sales (pop‑ups/market stall/pre‑order), iterate on feedback
  - Month 5: S